## GTSRB Sign Prediction

In [12]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import shutil
from sklearn.model_selection import train_test_split

In [29]:
import csv

In [13]:
def split_data(path_to_data, path_to_save_train, path_to_save_val, split_size=0.1):

    folders = os.listdir(path_to_data)

    for folder in folders:

        full_path = os.path.join(path_to_data, folder)
        images_paths = glob.glob(os.path.join(full_path, '*.png'))

        x_train, x_val = train_test_split(images_paths, test_size=split_size)

        for x in x_train:

            path_to_folder = os.path.join(path_to_save_train, folder)

            if not os.path.isdir(path_to_folder):
                os.makedirs(path_to_folder)

            shutil.copy(x, path_to_folder)

        for x in x_val:

            path_to_folder = os.path.join(path_to_save_val, folder)
            if not os.path.isdir(path_to_folder):
                os.makedirs(path_to_folder)

            shutil.copy(x, path_to_folder)

In [14]:
path_to_data = "C:\\Users\\\RASWANTH.SR\\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\Train"
path_to_save_train = "C:\\Users\\RASWANTH.SR\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\training_data\\train"
path_to_save_val = "C:\\Users\\RASWANTH.SR\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\training_data\\val"

In [15]:
split_data(path_to_data, path_to_save_train=path_to_save_train, path_to_save_val=path_to_save_val)

In [30]:
def order_test_set(path_to_images, path_to_csv):
    try:
        with open(path_to_csv, 'r') as csvfile:

            reader = csv.reader(csvfile, delimiter=',')

            for i, row in enumerate(reader):

                if i==0:
                    continue

                img_name = row[-1].replace('Test/', '')
                label = row[-2]

                path_to_folder = os.path.join(path_to_images, label)

                if not os.path.isdir(path_to_folder):
                    os.makedirs(path_to_folder)

                img_full_path = os.path.join(path_to_images, img_name)
                shutil.move(img_full_path, path_to_folder)

    except:
        print('[INFO] : Error reading csv file')

In [31]:
path_to_images = "C:\\Users\\RASWANTH.SR\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\Test"
path_to_csv = "C:\\Users\\RASWANTH.SR\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\Test.csv"

In [32]:
order_test_set(path_to_images, path_to_csv)

In [33]:
from tensorflow.keras.layers import Conv2D,Input,MaxPool2D,BatchNormalization,GlobalAvgPool2D,Dense,Flatten

In [74]:
def streets_sign_model(n_classes):
    my_input = Input(shape =(60,60,3))
    
    x = Conv2D(32,(3,3),activation = 'relu')(my_input) 
    x = Conv2D(64,(3,3),activation = 'relu')(x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)

    x = Conv2D(128,(3,3),activation = 'relu')(x)
    x = MaxPool2D()(x)
    x = BatchNormalization()(x)
    
    x = Flatten()(x)
    x = Dense(64,activation = 'relu')(x)
    x = Dense(n_classes, activation='softmax')(x)
    
    model = tensorflow.keras.Model(inputs = my_input,outputs=x)
    
    return model

In [75]:
model = streets_sign_model(10)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 conv2d_17 (Conv2D)          (None, 58, 58, 32)        896       
                                                                 
 conv2d_18 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 28, 28, 64)       0         
 g2D)                                                            
                                                                 
 batch_normalization_10 (Bat  (None, 28, 28, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_19 (Conv2D)          (None, 26, 26, 128)       7385

In [76]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [77]:
def create_generators(batch_size, train_data_path, val_data_path, test_data_path):

    preprocessor = ImageDataGenerator(
        rescale = 1 / 255.,
    )

    train_generator = preprocessor.flow_from_directory(
        train_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode='rgb',
        shuffle=True,
        batch_size=batch_size
    )

    val_generator = preprocessor.flow_from_directory(
        val_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode="rgb",
        shuffle=False,
        batch_size=batch_size,
    )

    test_generator = preprocessor.flow_from_directory(
        test_data_path,
        class_mode="categorical",
        target_size=(60,60),
        color_mode="rgb",
        shuffle=False,
        batch_size=batch_size,
    )

    return train_generator, val_generator, test_generator

In [78]:
path_to_train = "C:\\Users\\RASWANTH.SR\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\training_data\\train"
path_to_val = "C:\\Users\\RASWANTH.SR\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\training_data\\val"
path_to_test = "C:\\Users\\\RASWANTH.SR\\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\Test"
batch_size  = 64
epochs = 15

In [79]:
train_generator, val_generator, test_generator = create_generators(batch_size, path_to_train, path_to_val, path_to_test)

Found 35288 images belonging to 43 classes.
Found 3921 images belonging to 43 classes.
Found 12630 images belonging to 43 classes.


In [80]:
path_to_save_model = "./Models"
ckpt_saver = ModelCheckpoint(
    path_to_save_model,
    monitor = 'val_accuracy',
    mode ='max',
    save_best_only =True,
    save_freq='epoch',
    verbose =1
)

early_stop = EarlyStopping(monitor = 'val_accuracy', patience =10)

In [81]:
n_classes = train_generator.num_classes

model = streets_sign_model(n_classes)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_generator,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=val_generator,
                callbacks=[ckpt_saver, early_stop]
                )

Epoch 1/15
552/552 [==============================] - ETA: 0s - loss: 0.4998 - accuracy: 0.8789
Epoch 00001: val_accuracy improved from -inf to 0.83168, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
552/552 [==============================] - 262s 473ms/step - loss: 0.4998 - accuracy: 0.8789 - val_loss: 0.6312 - val_accuracy: 0.8317
Epoch 2/15
552/552 [==============================] - ETA: 0s - loss: 0.0545 - accuracy: 0.9844
Epoch 00002: val_accuracy improved from 0.83168 to 0.97016, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
552/552 [==============================] - 277s 501ms/step - loss: 0.0545 - accuracy: 0.9844 - val_loss: 0.1065 - val_accuracy: 0.9702
Epoch 3/15
552/552 [==============================] - ETA: 0s - loss: 0.0319 - accuracy: 0.9904
Epoch 00003: val_accuracy improved from 0.97016 to 0.97858, saving model to .\Models
INFO:tensorflow:Assets written to: .\Models\assets
552/552 [============================

In [83]:
model = tensorflow.keras.models.load_model('./Models')
model.summary()

print("Evaluating validation set:")
model.evaluate(val_generator)

print("Evaluating test set : ")
model.evaluate(test_generator)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 58, 58, 32)        896       
                                                                 
 conv2d_21 (Conv2D)          (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 28, 28, 64)       0         
 g2D)                                                            
                                                                 
 batch_normalization_12 (Bat  (None, 28, 28, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_22 (Conv2D)          (None, 26, 26, 128)       7385

[0.3578275740146637, 0.955898642539978]

In [84]:
import tensorflow as tf
def predict_with_model(model, imgpath):

    image = tf.io.read_file(imgpath)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.resize(image, [60,60]) # (60,60,3)
    image = tf.expand_dims(image, axis=0) # (1,60,60,3)

    predictions = model.predict(image) 
    predictions = np.argmax(predictions)

    return predictions

In [87]:
img_path = "C:\\Users\\RASWANTH.SR\\Desktop\\ML\\Neural Networks and Deep Learning\\Tensorflow for CV\\CSV\\Test\\28\\07827.png"
model = tf.keras.models.load_model('./Models')
prediction = predict_with_model(model, img_path)
print(f"prediction = {prediction}")

prediction = 21
